In [10]:
from openai import OpenAI
import ollama
from pydantic import BaseModel, Field
import requests
import json
import os

In [2]:
model = "llama3.1:8b"

In [3]:
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)

In [8]:
class NearbySearch(BaseModel):
    name: str = Field(description="The name of the place.")
    address: str = Field(description="Address of the place.")
    types: list[str] = Field(description="List of the types that the place is categorized as.") # Corrected typo
    place_id: str = Field(description="Unique id that is associated with the place")
    response: str = Field(description="A natural language reponse to the user.")

In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_nearby_places",
            "description": "Search for nearby places by a list of categories.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "categories": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "A list of categories to search for, e.g., ['restaurant', 'shopping_mall']"
                    }
                },
                "required": ["categories"],
            },
        },
    }
]

In [ ]:
from ./secerets import google_api

In [11]:
GOOGLE_API = google_api 

# Define valid categories once
VALID_CATEGORIES = [
    "restaurant", "food", "meal_takeaway", "meal_delivery", "cafe", "bakery",
    "bar", "night_club", "shopping_mall", "store", "gas_station", "hospital",
    "pharmacy", "bank", "atm", "tourist_attraction", "lodging", "park"
]

def search_nearby_places(categories: list[str]):
    """
    Searches for nearby places based on a list of categories.
    """
    if not categories:
        return json.dumps({"error": "Invalid call: no categories provided."}, indent=2)

    location = "27.7676,-82.6403"  # St. Petersburg, FL coordinates
    radius = 5000
    formatted_results = []

    # Loop through all provided categories
    for category in categories:
        if category not in VALID_CATEGORIES:
            print(f"Skipping invalid category: {category}")
            continue

        endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            'location': location,
            'radius': radius,
            'type': category,
            'key': GOOGLE_API
        }
        
        try:
            print(f"Searching for category: {category}...")
            response = requests.get(endpoint, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            status = data.get('status')
            
            if status == 'OK':
                results = data.get("results", [])
                # Add the top 2 results for this category to our list
                for place in results[:2]:
                    formatted_place = {
                        "category_searched": category,
                        "name": place.get("name"),
                        "vicinity": place.get("vicinity"),
                        "rating": place.get("rating"),
                        "types": place.get("types", []),
                    }
                    formatted_results.append(formatted_place)
            else:
                print(f"API Error for category '{category}': {data.get('error_message', status)}")

        except requests.exceptions.RequestException as e:
            print(f"Request failed for category '{category}': {e}")
            continue
            
    if not formatted_results:
        return json.dumps({"message": "No results found for the given categories."}, indent=2)
        
    return json.dumps(formatted_results, indent=2)

In [14]:
messages = [
    {"role": "system", "content": f"You are a helpful assistant that gathers accurate place recommendations. Allowed categories are {VALID_CATEGORIES}"},
    {"role": "user", "content": "where can i get some food?"}
]

In [16]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools
) 

In [17]:
response.model_dump()

{'id': 'chatcmpl-147',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': '',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_s49u7n70',
      'function': {'arguments': '{"categories":"[\\"food\\", \\"meal_takeaway\\", \\"meal_delivery\\"]"}',
       'name': 'search_nearby_places'},
      'type': 'function',
      'index': 0}]}}],
 'created': 1753714379,
 'model': 'llama3.1:8b',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_ollama',
 'usage': {'completion_tokens': 30,
  'prompt_tokens': 275,
  'total_tokens': 305,
  'completion_tokens_details': None,
  'prompt_tokens_details': None}}